Zero shot classification

In [8]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()  # Load environment variables from .env file, contains personal access token

API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-mnli"
headers = {"Authorization": f"Bearer {os.getenv('HF_API_TOKEN')}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Example usage
output = query({"inputs": "I love Python!", "parameters": {"candidate_labels": ["programming", "sports"]}})
print(output)


{'sequence': 'I love Python!', 'labels': ['programming', 'sports'], 'scores': [0.9925740957260132, 0.007425872143357992]}
